# Sentence embeddings
We will mainly use `sentence-transformers`, which is a dedicated package from Hugging Face 🤗. 

Relevant documentation
- Semantic textual similarity https://www.sbert.net/docs/usage/semantic_textual_similarity.html
- Semantic search https://www.sbert.net/examples/applications/semantic-search/README.html

In [ ]:
!pip install -U sentence-transformers faiss-cpu langchain langchain-community "unstructured[all-docs]" openai nest-asyncio streamlit jq

### From word embeddings to sentence embeddings

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Sentences we want to encode. Example:
sentence = ['This framework generates embeddings for each input sentence']

# Sentences are encoded by calling model.encode()
embedding = model.encode(sentence)
embedding

See, a sentence embedding is just a vector, just like a word embedding. That means we can also calculate similarities in a similar way:

In [ ]:

# Two lists of sentences
sentences1 = ['The cat sits outside',
             'A man is playing guitar',
             'The new movie is awesome!']

sentences2 = ['The dog plays in the garden',
              'My plants look a bit sick, could it be bitrot?',
              'The new movie is so great!']

#Compute embedding for both lists
embeddings1 = model.encode(sentences1, convert_to_tensor=True)
embeddings2 = model.encode(sentences2, convert_to_tensor=True)

#Compute cosine-similarities
cosine_scores = util.cos_sim(embeddings1, embeddings2)

#Output the pairs with their score
for i in range(len(sentences1)):
    print("{} \t\t {} \t\t Score: {:.4f}".format(sentences1[i], sentences2[i], cosine_scores[i][i]))

## Semantic search and retrieval

The idea behind semantic search is to embed all entries in your corpus, whether they be sentences, paragraphs, or documents, into a vector space.

At search time, the query is embedded into the same vector space and the closest embeddings from your corpus are found. These entries should have a high semantic overlap with the query.


![title](https://raw.githubusercontent.com/UKPLab/sentence-transformers/master/docs/img/SemanticSearch.png
)

Instead of trying to build a semantic search engine from first principles, we'll use `langchain`. 

## [Don't run this again] Crawl the Vlerick website using Apify

The following code crawls the Vlerick website so we have some text to model. It's just example code. 

Langchain supports more than 100 integrations, so depending on where you find interesting data you'll need to use something else.

In [ ]:
# from langchain.utilities import ApifyWrapper
# import os

# os.environ["APIFY_API_TOKEN"] = ""

# apify = ApifyWrapper()
# # Call the Actor to obtain text from the crawled webpages
# loader = apify.call_actor(
#     actor_id="apify/website-content-crawler",
#     run_input={
#         "startUrls": [{"url": "https://www.vlerick.com/en/"}]
#     },
#     dataset_mapping_function=lambda item: Document(
#         page_content=item["text"] or "", metadata={"source": item["url"]}
#     ),
# )


## Create new vector store and embed all documents
Source: https://python.langchain.com/docs/expression_language/cookbook/retrieval

In [ ]:
# Let's load all documents
# Adapt this code to your own source of data.

from langchain_community.document_loaders import DirectoryLoader
from pathlib import Path
from pprint import pprint

from langchain.document_loaders import TextLoader, DirectoryLoader, JSONLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
%time

loader = DirectoryLoader('/Users/jospolfliet/src/vlerick/DATA/MAI-2023 dump/', silent_errors=True)
course_docs = loader.load()

print(f"Number of documents {len(course_docs)}")

In [ ]:
course_docs[9:12]

In [ ]:

from langchain_community.document_loaders import ApifyDatasetLoader
from langchain_community.document_loaders.base import Document

loader = ApifyDatasetLoader(
    dataset_id="RcArHfVs80xOg9IKs",
    dataset_mapping_function=lambda dataset_item: Document(
        page_content=dataset_item["text"], metadata={"source": dataset_item["url"]}
    ),
)
website_docs = loader.load()
print(f"Number of documents {len(website_docs)}")
website_docs = [doc for doc in website_docs if not doc.page_content.startswith("Your choice regarding cookies on this site")]
print(f"Number of non-trivial documents {len(website_docs)}")
website_docs[5:7]

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(course_docs + website_docs)
documents[0]
print(f"Number of chunks {len(documents)}")

### Embed into a vector store - and cache the results
We got a decent store of data loaded into memory now. Next thing we need to do is calculate sentence embeddings. 
We'll use simple, reasonably fast embeddings that we can calculate locally withouting requiring an expensive GPU or cloud service like OpenAI's GPTx.

In [ ]:
%%time

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
# to test, use query_result = embeddings.embed_query("My text")

if True: # change to True if you want to (re)create your store   
    vectorstore = FAISS.from_documents(
        documents, embedding=embeddings
    )
    # store because this is slow
    vectorstore.save_local("vectorstore") 

In [ ]:
vectorstore = FAISS.load_local("vectorstore", embeddings)
vectorstore.index

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.chat_models import ChatOpenAI
from operator import itemgetter

retriever = vectorstore.as_retriever(k=8)


In [ ]:
def q(s):
    results = retriever.get_relevant_documents(s)
    for doc in results:
        print("#"*100)
        print(doc.metadata["source"])
        print("#"*100)
        print(doc.page_content)
q("stochastic gradient descent")

In [ ]:
q("what type of awards does vlerick give")